In [1]:
import os
MODEL = 'llama3.1'
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
model=Ollama(model=MODEL)
embeddings= OllamaEmbeddings(model=MODEL)

In [2]:
model.invoke('tell me a short joke')

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta."

**Loading libraries**

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys

In [4]:
loader = CSVLoader(file_path="C:/Users/ASUS/Desktop/local_model/csv_RAG/data/Employees.csv", encoding="utf-8", csv_args={'delimiter': ';'})

In [5]:
import pandas as pd
x=pd.read_csv("C:/Users/ASUS/Desktop/local_model/csv_RAG/data/Employees.csv",delimiter=';')

In [6]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   No              689 non-null    int64 
 1   First Name      689 non-null    object
 2   Last Name       689 non-null    object
 3   Gender          689 non-null    object
 4   Start Date      689 non-null    object
 5   Years           689 non-null    int64 
 6   Department      689 non-null    object
 7   Country         689 non-null    object
 8   Center          689 non-null    object
 9   Monthly Salary  689 non-null    int64 
 10  Annual Salary   689 non-null    int64 
 11  Job Rate        689 non-null    object
 12  Sick Leaves     689 non-null    int64 
 13  Unpaid Leaves   689 non-null    int64 
 14  Overtime Hours  689 non-null    int64 
dtypes: int64(7), object(8)
memory usage: 80.9+ KB


In [7]:
data = loader.load()
print(data)

[Document(page_content='No: 1\nFirst Name: Ghadir\nLast Name: Hmshw\nGender: Male\nStart Date: 04/04/2018\nYears: 6\nDepartment: Quality Control\nCountry: Egypt\nCenter: West\nMonthly Salary: 1560\nAnnual Salary: 18720\nJob Rate: 3\nSick Leaves: 1\nUnpaid Leaves: 0\nOvertime Hours: 183', metadata={'source': 'C:/Users/ASUS/Desktop/local_model/csv_RAG/data/Employees.csv', 'row': 0}), Document(page_content='No: 2\nFirst Name: Omar\nLast Name: Hishan\nGender: Male\nStart Date: 21/05/2020\nYears: 4\nDepartment: Quality Control\nCountry: Saudi Arabia\nCenter: West\nMonthly Salary: 3247\nAnnual Salary: 38964\nJob Rate: 1\nSick Leaves: 0\nUnpaid Leaves: 5\nOvertime Hours: 198', metadata={'source': 'C:/Users/ASUS/Desktop/local_model/csv_RAG/data/Employees.csv', 'row': 1}), Document(page_content='No: 3\nFirst Name: Ailya\nLast Name: Sharaf\nGender: Female\nStart Date: 28/09/2017\nYears: 6\nDepartment: Major Mfg Projects\nCountry: Saudi Arabia\nCenter: West\nMonthly Salary: 2506\nAnnual Salary: 3

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

print(len(text_chunks))


689


In [9]:
print(text_chunks[0].page_content)


No: 1
First Name: Ghadir
Last Name: Hmshw
Gender: Male
Start Date: 04/04/2018
Years: 6
Department: Quality Control
Country: Egypt
Center: West
Monthly Salary: 1560
Annual Salary: 18720
Job Rate: 3
Sick Leaves: 1
Unpaid Leaves: 0
Overtime Hours: 183


In [10]:

# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
# Converting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)

In [12]:
#saving database
DB_FAISS_PATH = "C:/Users/ASUS/Desktop/local_model/csv_RAG/vectorstore/db_faiss"
docsearch.save_local(DB_FAISS_PATH)

In [13]:
query="talk about 5 employees from our quality control departement and how much is their average pay"
docs = docsearch.similarity_search(query,k=10)
print("Result", docs)
print(docs[0].page_content)

Result [Document(page_content='No: 169\nFirst Name: Majd\nLast Name: Yasin\nGender: Female\nStart Date: 16/04/2019\nYears: 5\nDepartment: Quality Control\nCountry: United Arab Emirates\nCenter: South\nMonthly Salary: 2196\nAnnual Salary: 26352\nJob Rate: 5\nSick Leaves: 4\nUnpaid Leaves: 0\nOvertime Hours: 9', metadata={'source': 'C:/Users/ASUS/Desktop/local_model/csv_RAG/data/Employees.csv', 'row': 168}), Document(page_content='No: 115\nFirst Name: Mahir\nLast Name: Alshear\nGender: Male\nStart Date: 02/06/2019\nYears: 5\nDepartment: Quality Control\nCountry: Egypt\nCenter: North\nMonthly Salary: 3166\nAnnual Salary: 37992\nJob Rate: 4,5\nSick Leaves: 5\nUnpaid Leaves: 0\nOvertime Hours: 4', metadata={'source': 'C:/Users/ASUS/Desktop/local_model/csv_RAG/data/Employees.csv', 'row': 114}), Document(page_content='No: 418\nFirst Name: Jury\nLast Name: Eabuwd\nGender: Female\nStart Date: 04/12/2018\nYears: 5\nDepartment: Quality Control\nCountry: Egypt\nCenter: South\nMonthly Salary: 979\n

In [14]:

from langchain.prompts import PromptTemplate

template = """Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [15]:
retriever = docsearch.as_retriever(search_kwargs={"k": 10})

In [16]:
from langchain.prompts import PromptTemplate

from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
)

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [17]:
answer=chain.invoke({'question':'talk about 5 employees from our quality control departement and how much is their average pay'})

In [18]:
print(answer)


Based on the provided context, I can identify 3 employees from the Quality Control department. Here are their details:

1. **Majd Yasin** (No: 169): Monthly Salary = 2196, Annual Salary = 26352
2. **Hasan Kalthum** (No: 275): Monthly Salary = 831, Annual Salary = 9972
3. **Iad Badran** (No: 600) is not from Quality Control but from Quality Assurance department.
However, I can identify another one which is 

4. **Shuruq Aljuju** (No: 605): Monthly Salary = 1573, Annual Salary = 18876
5. **Hasan Quidr** (No: 281): Monthly Salary = 2488, Annual Salary = 29856

The average pay of these 4 employees is:
((2196 + 831 + 1573 + 2488) / 4) ≈ 1722

Note that I excluded **Iad Badran** as he is not from Quality Control department.


In [19]:
answer=chain.invoke({'question':'are there any employees from syria if so what are their first names, and is there ahmad among them ?'})

In [20]:
print(answer)

Yes, there are employees from Syria. Some of the first names mentioned for employees from Syria are:

- Ahmad
- Hasan
- Ayham (twice)
- Muhamad (twice)
- Hamdah
- Abdalkarim
- Sarih 

So yes, Ahmad is among them.


In [21]:
x[x['No']==258]

,No,First Name,Last Name,Gender,Start Date,Years,Department,Country,Center,Monthly Salary,Annual Salary,Job Rate,Sick Leaves,Unpaid Leaves,Overtime Hours
257,258,Ahmad,Alshalaq,Male,05/04/2019,5,Product Development,Syria,North,983,11796,5,0,0,13


In [22]:
answer=chain.invoke({'question':'how many total leaves did Iad Badran take how many of them where unpaid, also how much overtime hours did he work?'})

In [23]:
print(answer)

Based on the context, here are the answers to your question:

* Total sick leaves taken by Iad Badran: 6
* Unpaid leaves taken by Iad Badran: 5
* Overtime hours worked by Iad Badran: 8


In [24]:
answer=model.invoke('''create a simple pandas script (python) to calculate 
                    the average (Annual Salary) of people where their (Country) is (Egypt) from 
                    a data frame (x) and return the answer in a string ,
                    the dataset exists already so only give the code to the necessary
                    parts,also give only code and not answer so it would exec directly your msg''')

In [25]:
print(answer)

```python
average_annual_salary = x[x['Country'] == 'Egypt']['Annual Salary'].mean().round(2)
print(f'The average annual salary in Egypt is: ${average_annual_salary}')
```


In [26]:
def reform(answer):
    parsed_answer=answer.split('```')
    if len(parsed_answer)>1:
        code=parsed_answer[1]
        exec(code[6:])
    else:
        print(parsed_answer[0])

In [27]:
reform(answer)

The average annual salary in Egypt is: $25078.8


In [33]:
from langchain.prompts import PromptTemplate

from operator import itemgetter

from langchain.prompts import PromptTemplate

template = """Answer the question based on the context provided below. If the answer can be directly inferred from the context, provide it in a clear and concise manner. If complex computations are required, or if the info provided in the context are not enough, generate a simple Python script using Pandas to calculate the answer.

Structure your response as follows:

- For direct answers from the context:
  Example:
  "The question is 'Where is Ahmad Khaled from?'
  The answer is:
  Ahmad Khaled is from Italy."

- For answers requiring code:
  Provide the code in a single block with a brief explanation.
  Example:
  "The question is 'What is the average annual salary of people from Egypt?'
  The answer is:
  ```python
  import pandas as pd
  average_salary = round(x[x['Country'] == 'Egypt']['Annual Salary'].mean(), 2)
  print('The average salary in Egypt is: average_salary')
  ```

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
newchain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
)

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

Answer the question based on the context provided below. If the answer can be directly inferred from the context, provide it in a clear and concise manner. If complex computations are required, or if the info provided in the context are not enough, generate a simple Python script using Pandas to calculate the answer.

Structure your response as follows:

- For direct answers from the context:
  Example:
  "The question is 'Where is Ahmad Khaled from?'
  The answer is:
  Ahmad Khaled is from Italy."

- For answers requiring code:
  Provide the code in a single block with a brief explanation.
  Example:
  "The question is 'What is the average annual salary of people from Egypt?'
  The answer is:
  ```python
  import pandas as pd
  average_salary = round(x[x['Country'] == 'Egypt']['Annual Salary'].mean(), 2)
  print('The average salary in Egypt is: average_salary')
  ```

Context: Here is some context

Question: Here is a question



In [ ]:
from langchain.prompts import PromptTemplate

from operator import itemgetter

from langchain.prompts import PromptTemplate

template = """Answer the question based on the context below. if there is complex computations or if the 10 closest users
related to it (context) are not enough create a simple pandas script (python) that would return the answer in a string,
our dataframe (x) already declared
if you give code write it all in a single block and not multiple ones so answer would be like this "
if the question is how many
```python
print(len(x))
```
"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
newchain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
)

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

Answer the question based on the context below. if there is complex computations or if the 10 closest users
related to it (context) are not enough create a simple pandas script (python) that would return the answer in a string,
our dataframe (x) already declared
if you give code write it all in a single block and not multiple ones so answer would be like this "
if the question is how many
```python
print(len(x))
```
"

Context: Here is some context

Question: Here is a question



In [ ]:
answer=newchain.invoke({'question':'how many users do we have'})

In [ ]:
print(answer)

Here is the answer:

your question is how many users do we have,
the answer is 
```python
print(len(x))
```
Note that I assumed `x` is your pandas dataframe, as mentioned in the context. Let me know if you need further assistance!


In [ ]:
reform(answer)

689


In [ ]:
answer=newchain.invoke({'question':'how many users from each country work in our company'})
reform(answer)

Country
Saudi Arabia            3
Egypt                   2
Syria                   1
United Arab Emirates    1
Name: count, dtype: int64


In [31]:
def llm(question,origin=0):
    answer=newchain.invoke({'question':question})
    reform(answer)
    if origin:
        print('the original answer is ')
        print()
        print(answer)

In [ ]:
llm('give me an annual salary value example ',1)

54666
the original answer is 

Your question is "give me an annual salary value example"
The answer is 
```python
print(54666)
```


In [ ]:
llm('whats the first name of the most paid employee',1)

Ashraf
the original answer is 

Here is the answer:

"your question is whats the first name of the most paid employee,
the answer is 
```python
print(x.loc[x['Annual Salary'].idxmax()]['First Name'])
```
"


In [34]:
llm('how many total leaves did Iad Badran take, how many of them were unpaid, also how much overtime hours did he work?',1)

Iad Badran took a total of 11 leaves.
Of those, 5 were unpaid.
Iad Badran worked a total of 8 overtime hours.
the original answer is 

The answer is:
```python
import pandas as pd

data = {
    "First Name": ["Iad", "Ahmad", "Iad", "Ahmad", "Samir", "Amani", "Sultan", "Ahmad", "Basmah", "Ahmad"],
    "Last Name": ["Badran", "Eurman", "Alhusayn", "Swyd", "Almisri", "Altujjar", "Alhijar", "Almisri", "Alhabis", "Aldwltali"],
    "Country": ["Egypt", "Egypt", "Egypt", "Egypt", "Egypt", "Egypt", "Egypt", "United Arab Emirates", "Egypt", "United Arab Emirates"],
    "Sick Leaves": [6, 0, 2, 1, 1, 0, 2, 0, 0, 0],
    "Unpaid Leaves": [5, 0, 6, 0, 0, 1, 0, 0, 0, 0],
    "Overtime Hours": [8, 5, 7, 7, 8, 7, 45, 1, 3, 35]
}

df = pd.DataFrame(data)

print(f"Iad Badran took a total of {df.loc[df['First Name'] == 'Iad', 'Sick Leaves'].values[0] + df.loc[df['First Name'] == 'Iad', 'Unpaid Leaves'].values[0]} leaves.")
print(f"Of those, {df.loc[df['First Name'] == 'Iad', 'Unpaid Leaves'].values[0]} 